In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install hf_transfer



In [ ]:
!nvidia-smi

## Gemma - 7B

In [3]:
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import os


os.environ ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'

In [5]:
torch.__version__

'2.2.0+cu121'

In [6]:
import os

os.environ ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'

In [7]:
torch.device("cuda")

device(type='cuda')

In [8]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

model = AutoModelForCausalLM.from_pretrained("google/gemma-7b-it", # 'google/gemma-2b-it
                                             quantization_config=quantization_config,
                                             low_cpu_mem_usage=True,
                                             torch_dtype="auto",
                                             device_map="auto"
                                             )

tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
tokenizer.vocab_size

256000

In [10]:
chat = [
    { "role": "user", "content": "What is the difference between LLaMAs, Alpacas, and Vicunas" },
]

prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
prompt


'<start_of_turn>user\nWhat is the difference between LLaMAs, Alpacas, and Vicunas<end_of_turn>\n<start_of_turn>model\n'

In [11]:
inputs = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt")

outputs = model.generate(input_ids=inputs.to(model.device),
                         max_new_tokens=512)

In [14]:
from IPython.display import Markdown, display
display(Markdown("## Gemma - 7B 4Bit"))

text = tokenizer.decode(outputs[0],skip_special_tokens=True, clean_up_tokenization_spaces=True)
# text = text.replace(prompt, '', 1)
display(Markdown(text))

## Gemma - 7B 4Bit

user
What is the difference between LLaMAs, Alpacas, and Vicunas
model
Sure, here is the difference between LLamas, Alpacas, and Vicunas:

**LLamas:**

* **Origin:** North America
* **Coat:** Long, dense, soft, and warm
* **Color:** Gray, brown, black, white
* **Temperament:** Friendly, calm, and easy to train
* **Uses:** Wool production, meat production, and companionship

**Alpacas:**

* **Origin:** South America
* **Coat:** Long, dense, soft, and warm
* **Color:** White, fawn, gray, brown
* **Temperament:** Friendly, calm, and intelligent
* **Uses:** Wool production, meat production, and companionship

**Vicunas:**

* **Origin:** South America
* **Coat:** Long, dense, soft, and warm
* **Color:** White, fawn, gray, brown
* **Temperament:** Friendly, calm, and intelligent
* **Uses:** Wool production, meat production, and companionship

The main differences between LLamas, Alpacas, and Vicunas are their origin, coat color, temperament, and uses. LLamas are North American animals, while Alpacas and Vicunas are South American animals. LLamas have a wide range of coat colors, while Alpacas and Vicunas have a more limited range of coat colors. LLamas are generally friendly and easy to train, while Alpacas and Vicunas are also friendly and easy to train. LLamas are primarily used for wool production and meat production, while Alpacas and Vicunas are primarily used for wool production and meat production.